In [12]:
import pandas as pd
from sqlalchemy import create_engine
import mysql.connector
import click
from flask import current_app, g
from flask.cli import with_appcontext
import os


In [23]:

def getDB():
    db = mysql.connector.connect(
        host = "localhost",
        user = "root",
        password = "",
        database = "apra_etl",
        connect_timeout = 60
    )
    c = db.cursor(dictionary=True)
    return db, c


In [13]:
def abrev(text):
    # Split the text into words
    words = text.split()
    
    # Extract the first letter of each word
    initials = [word[0] for word in words]
    
    # Join the initials together to form the abbreviation
    abbreviation = ''.join(initials).upper()
    
    return abbreviation


In [29]:
folder = "./data/monthly/"
files = os.listdir(folder)
files = [f for f in files]

filename = files[0]
df = pd.read_excel(os.path.join(folder,filename), sheet_name='Table 1', skiprows=1)

In [34]:
columns = df.columns


i = 0
for col in columns:
    if len(col.split()) > 1:
        # Generate an abbreviation for the column name
        abbr = abrev(col)
        if abbr in df.columns:
            i += 1
            abbr += f"_{i}"
            df.rename(columns={col: abbr}, inplace=True)
        else:
            # Update the DataFrame with the abbreviated column name
            df.rename(columns={col: abbr}, inplace=True)

# db, c = getDB()

In [35]:
try:
    engine = create_engine('mysql+mysqlconnector://root@127.0.0.1/apra_etl', connect_args={'connect_timeout': 120})

    df.to_sql('datos_mensuales', con=engine, if_exists='replace', chunksize=1000)
except Exception as e:
    print(e)